# imports

In [2]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import pickle
import qplib as qp
from qplib import log, na, nk, num



# log()

In [3]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import qplib as qp
from qplib import log
pd.set_option('display.max_columns', None)



log('trace: this is a trace message')
log('debug: this is a debug message')
log('info: this is an info message')
log('warning: this is a warning message')
log('error: this is an error message')



0,this is a trace message,,trace,2024-07-25 10:49:44.025507


1,this is a debug message,,debug,2024-07-25 10:49:44.637035


2,this is an info message,,info,2024-07-25 10:49:44.641003


3,this is a warning message,,Warning,2024-07-25 10:49:44.644741


4,this is an error message,,ERROR,2024-07-25 10:49:44.650215


# pd_query

In [38]:

import numpy as np
import pandas as pd
import copy
import re
import qplib as qp

from IPython.display import display
from ipywidgets import widgets, interactive_output, HBox, VBox, fixed, Layout, interact_manual

from qplib.util import log
from qplib.types import qp_int, qp_float, qp_num, qp_bool, qp_datetime, qp_date, qp_na, qp_nk, qp_yn, qpDict
from qplib.pd_util import _check_df, _show_differences, _format_df, indexQpExtension, seriesQpExtension, dfQpExtension







class Symbol:
    def __init__(self, symbol, description, unary=False, binary=False, function=None, type=None, compatible=None):
        #default symbol = None
        self.symbol = symbol
        self.description = description
        self.unary = unary
        self.binary = binary
        self.function = function
        self.type = type
        self.compatible = compatible
    def __repr__(self):
        return f"Symbol('{self.symbol}': '{self.description})'"

class Symbols:   
    COMMENT = Symbol('#', 'comments out the rest of the line', type='comment',)
    ESCAPE = Symbol('`', 'escape the next character', type='escape',)



    #there are 4 different types of instructions
    SETTINGS = Symbol('´s', 'settings for the query', function='settings', type='instruction')
    SELECT_COLS = Symbol('´c', 'default. select columns by a condition', function='select_cols', type='instruction')
    SELECT_ROWS = Symbol('´r', 'select rows by a condition', function='select_rows', type='instruction')
    MODIFY_VALS = Symbol('´m', 'modify the currently selected values', function='modify_vals', type='instruction')
    NEW_COL = Symbol('´n', 'add a new column', function='new_col', type='instruction')
    TYPES = [SETTINGS, SELECT_COLS, SELECT_ROWS, MODIFY_VALS, NEW_COL]


    #select conditions can be combined with the previous conditions
    CONNECTOR_RESET = Symbol('', 'default. only the current condition must be fulfilled', type='connector')
    CONNECTOR_AND =   Symbol('&', 'this condition and the previous condition/s must be fulfilled', type='connector')
    CONNECTOR_OR =    Symbol('/', 'this condition or the previous condition/s must be fulfilled', type='connector')

    #when filtering rows, what should the condition apply to
    COLS_ANY = Symbol('any', 'default. any of the currently selected columns must fulfill the condition', type='flag')
    COLS_ALL = Symbol('all', 'all of the currently selected columns must fulfill the condition', type='flag')
    INDEX = Symbol('idx', 'the index of the dataframe must fulfill the condition', type='flag')

    #negation of selection conditions
    NEGATE_FALSE = Symbol('', 'default. dont negate the condition', type='flag')
    NEGATE_TRUE = Symbol('!', 'negate the condition', type='flag')



    #binary operators for filtering

    OP_BIGGER_EQUAL = Symbol('>=', 'bigger or equal', binary=True, type='operator')
    OP_SMALLER_EQUAL = Symbol('<=', 'smaller or equal', binary=True, type='operator')
    OP_BIGGER = Symbol('>', 'bigger', binary=True, type='operator')
    OP_SMALLER = Symbol('<', 'smaller', binary=True, type='operator')

    OP_STRICT_EQUAL = Symbol('==', 'strict equal', binary=True, type='operator')
    OP_EQUAL = Symbol('=', 'default. equal', binary=True, type='operator')

    OP_REGEX_MATCH = Symbol('~~', 'regex match', binary=True, type='operator')
    OP_REGEX_SEARCH = Symbol('~', 'regex search', binary=True, type='operator')

    OP_STRICT_CONTAINS = Symbol('(())', 'strict contains', binary=True, type='operator')
    OP_CONTAINS = Symbol('()', 'contains', binary=True, type='operator')

    OP_X_EVAL = Symbol('x?', 'select values by evaluating a python expression on each value', binary=True, type='operator')
    OP_COL_EVAL = Symbol('col?', 'select rows by evaluating a python expression on a whole column', binary=True, type='operator')
    
    OP_LOAD = Symbol('@', 'load a saved selection', binary=True, type='operator')


    #unary operators for filtering

    OP_IS_STR = Symbol('is str', 'is string', unary=True, type='operator')
    OP_IS_INT = Symbol('is int', 'is integer', unary=True, type='operator')
    OP_IS_FLOAT = Symbol('is float', 'is float', unary=True, type='operator')
    OP_IS_NUM = Symbol('is num', 'is number', unary=True, type='operator')
    OP_IS_BOOL = Symbol('is bool', 'is boolean', unary=True, type='operator')

    OP_IS_DATETIME = Symbol('is datetime', 'is datetime', unary=True, type='operator')
    OP_IS_DATE = Symbol('is date', 'is date', unary=True, type='operator')

    OP_IS_ANY = Symbol('is any', 'is any value', unary=True, type='operator')
    OP_IS_NA = Symbol('is na', 'is missing value', unary=True, type='operator')
    OP_IS_NK = Symbol('is nk', 'is not known value', unary=True, type='operator')
    OP_IS_YN = Symbol('is yn', 'is yes or no value', unary=True, type='operator')
    OP_IS_YES = Symbol('is yes', 'is yes value', unary=True, type='operator')
    OP_IS_NO = Symbol('is no', 'is no value', unary=True, type='operator')
    OP_IS_UNIQUE = Symbol('is unique', 'is unique value', unary=True, type='operator')



    #binary operators for modifying values
    OP_SET_VAL = Symbol('=', 'default. change currently selected values to the provided string', binary=True, type='operator')
    OP_ADD_VAL = Symbol('+=', 'add str to currently selected values (they are coerced to string)', binary=True, type='operator')

    OP_SET_X_EVAL = Symbol('x=', 'change values by evaluating a python expression for each currently selected value', binary=True, type='operator')
    OP_SET_COL_EVAL = Symbol('col=', 'change values by evaluating a python expression for each currently selected column', binary=True, type='operator')
    OP_SET_HEADER_EVAL = Symbol('header=', 'change headers (col names) by evaluating a python expression for each currently selected header', binary=True, type='operator')

    #unary operators for modifying values
    OP_TO_STR = Symbol('to str', 'convert currently selected values to string', unary=True, type='operator')
    OP_TO_INT = Symbol('to int', 'convert currently selected values to integer', unary=True, type='operator')
    OP_TO_FLOAT = Symbol('to float', 'convert currently selected values to float', unary=True, type='operator')
    OP_TO_NUM = Symbol('to num', 'convert currently selected values to number', unary=True, type='operator')
    OP_TO_BOOL = Symbol('to bool', 'convert currently selected values to boolean', unary=True, type='operator')

    OP_TO_DATETIME = Symbol('to datetime', 'convert currently selected values to datetime', unary=True, type='operator')
    OP_TO_DATE = Symbol('to date', 'convert currently selected values to date', unary=True, type='operator')

    OP_TO_NA = Symbol('to na', 'convert currently selected values to missing value', unary=True, type='operator')
    OP_TO_NK = Symbol('to nk', 'convert currently selected values to not known value', unary=True, type='operator')
    OP_TO_YN = Symbol('to yn', 'convert currently selected values to yes or no value', unary=True, type='operator')


    #binary operators for modifying the whole dataframe
    OP_NEW_COL_STR = Symbol('=', 'add a new string column to the dataframe and select it instead of current selection', binary=True, type='operator')
    OP_NEW_COL_TRUE = Symbol('@', 'add a new boolean column and select it. all currently selected rows are set to True', binary=True, type='operator')

class Expression:
    def __init__(self, text, function, line_num):
        self.text = text
        self.function = function
        self.line_num = line_num
        self.frepr = f'Expression({self.text})'
    def __repr__(self):
        return f'Instruction({self.__dict__})'


@pd.api.extensions.register_dataframe_accessor('q')
class DataFrameQuery:
    """
    """

    def __init__(self, df: pd.DataFrame):
        _check_df(df)
        self.df_og = df
        self.symbols = Symbols()

    def __repr__(self):
        return 'docstring of dataframe accessor pd_object.q():' + self.__doc__
    
    def __call__(self,
            code='',  #code in string form for filtering and modifying data
            inplace=True,  #make modifications inplace or just return a new dataframe.
            verbosity=3,  #verbosity level for logging. 0: no logging, 1: errors, 2: warnings, 3: info, 4: debug
            diff=None,  #[None, 'mix', 'old', 'new', 'new+']
            diff_max_cols=200,  #maximum number of columns to display when using diff. None: show all
            diff_max_rows=20,  #maximum number of rows to display when using diff. None: show all
            **kwargs
            ):
        
        #setup

        self.code = code
        self.inplace = inplace
        self.verbosity = verbosity
        self.diff = diff
        self.diff_max_cols = diff_max_cols
        self.diff_max_rows = diff_max_rows

        if inplace is False:
            self.df = self.df_og.copy()
        else:
            self.df = self.df_og  
        self.df.qp = self.df_og.qp 

        self.cols_filtered = pd.Index([True for col in self.df.columns])
        self.rows_filtered = pd.Index([True for row in self.df.index])



        #main

        self.parse(self.code)

        for expression in self.expressions:
            expression.symbol = expression.text[:2]
            expression.type, text_temp = self.match_symbol(expression.text, None, self.symbols.TYPES)
            expression.function = expression.type.function
            if expression.type is None:
                log(f'warning: no defined behaviour for "{expression.symbol}"', 'df.q()', self.verbosity)
                continue
    
            self.expression = expression

            #parse and evaluate the expression
            self.__getattribute__(expression.function)()



        #results

        self.df_filtered = self.df.loc[self.rows_filtered, self.cols_filtered]
        self.df_filtered.qp = self.df.qp
        self.df_filtered.qp.code = self.code
    
        if self.diff is None:
            return self.df_filtered 
        else:
            #show difference before and after filtering

            if 'meta' in self.df.columns and 'meta' not in self.df_filtered.columns:
                self.df_filtered.insert(0, 'meta', self.df.loc[self.rows_filtered, 'meta'])

            result = _show_differences(
                self.df_filtered, self.df, show=self.diff,
                max_cols=self.diff_max_cols, max_rows=self.diff_max_rows,
                verbosity=self.verbosity)  
            return  result  
   

    def parse(self, code):
        self.lines = []
        self.expressions = []
        self.instructions = []

        #get lines and expression blocks
        for line_num, line in enumerate(self.code.split('\n')):
            line = line.strip()
            self.lines.append([line_num, line])
            line = line.split(self.symbols.COMMENT.symbol)[0].strip()
        
            if line == '':
                continue


            AND = self.symbols.CONNECTOR_AND.symbol
            OR = self.symbols.CONNECTOR_OR.symbol

            escape = False
            chars_in_expression = 0
            expression_type = self.symbols.SELECT_COLS.symbol  #default

            for i, char in enumerate(line):
                if escape:
                    self.expressions[-1].text += char
                    chars_in_expression += 1
                    escape = False
                    continue
                elif char == self.symbols.ESCAPE.symbol:
                    escape = True
                    continue

                if char == '´':
                    expression_type = char + line[i+1]
                    self.expressions.append(Expression(char, None, line_num))
                    chars_in_expression = 1
                elif char in [AND, OR]:
                    if chars_in_expression >= 3:
                        self.expressions.append(Expression(f'{expression_type} {char}', None, line_num))
                        chars_in_expression = 3
                    elif i == 0:
                        self.expressions.append(Expression(f'{expression_type} {char}', None, line_num))
                        chars_in_expression = 3
                    else:
                        self.expressions[-1].text += char
                        chars_in_expression += 1
                elif i == 0:
                    self.expressions.append(Expression(f'{expression_type} {char}', None, line_num))
                    chars_in_expression = 3
                elif char == ' ':
                    self.expressions[-1].text += char
                else:
                    self.expressions[-1].text += char
                    chars_in_expression += 1

    def match_symbol(self, string, default, symbols):
        string = string.strip()

        for symbol in symbols:
            if string.startswith(symbol.symbol):
                log(f'trace: found symbol "{symbol}" in string "{string}"', '_read_symbol', self.verbosity)
                return symbol, string[len(symbol.symbol):].strip()
        
        log(f'trace: no symbol found in string "{string}". using default "{default}"', '_read_symbol', self.verbosity)
        
        if default is None:
            return None, string
        if string.startswith(default.symbol):
            string = string[len(default.symbol):].strip()
        return default, string


    def settings(self):

        text = self.expression.text[2:].strip()

        setting = None
        for setting_temp in ['verbosity', 'diff', 'diff_max_cols', 'diff_max_rows']:
            if text.startswith(setting_temp):
                setting = setting_temp
                text = text[len(setting_temp):].strip()
                break
        
        if setting is None:
            log(f'warning: no setting found in "{self.expression.text}"', 'df.q.settings()', self.verbosity)
            return
        
        if text[0] != '=':
            log(f'warning: settings can only be set with "=" not with "{text[0]}"', 'df.q.settings()', self.verbosity)
            return
        else:
            value = text[1:].strip()


        if setting == 'verbosity':
            if value in ['0', '1', '2', '3', '4', '5']:
                self.verbosity = int(value)
            else:
                log(f'warning: verbosity must be an integer between 0 and 5. "{value}" is not valid',
                    'df.q.settings()', self.verbosity)
        
        elif setting == 'diff':
            if value in ['none', 'None', 'NONE', '0']:
                self.diff = None
            elif value.lower() in ['mix', 'new', 'old', 'new+']:
                self.diff = value.lower()
            else:
                log(f'warning: diff must be one of [None, "mix", "old", "new", "new+"]. "{value}" is not valid',
                    'df.q.settings()', self.verbosity)
                
        elif setting == 'diff_max_cols':
            if value in ['none', 'None', 'NONE', '0']:
                self.diff_max_cols = None
            try:
                self.diff_max_cols = int(value)
            except:
                log(f'warning: diff_max_cols must be an integer or None. "{value}" is not valid',
                    'df.q.settings()', self.verbosity)
                
        elif setting == 'diff_max_rows':
            if value in ['none', 'None', 'NONE', '0']:
                self.diff_max_rows = None
            try:
                self.diff_max_rows = int(value)
            except:
                log(f'warning: diff_max_rows must be an integer or None. "{value}" is not valid',
                    'df.q.settings()', self.verbosity)


    def select_cols(self):

        #parse the expression

        connector, text = self.match_symbol(
            self.expression.text[2:],
            default=self.symbols.CONNECTOR_RESET,
            symbols=[
                self.symbols.CONNECTOR_AND,
                self.symbols.CONNECTOR_OR
                ]
            )

     
        negate, text = self.match_symbol(
            text,
            default=self.symbols.NEGATE_FALSE,
            symbols=[self.symbols.NEGATE_TRUE]
            )
        
        operator, text = self.match_symbol(
            text,
            default=self.symbols.OP_EQUAL,
            symbols=[
                self.symbols.OP_BIGGER_EQUAL,
                self.symbols.OP_SMALLER_EQUAL,
                self.symbols.OP_BIGGER,
                self.symbols.OP_SMALLER,
                self.symbols.OP_STRICT_EQUAL,
                self.symbols.OP_EQUAL,
                self.symbols.OP_REGEX_MATCH,
                self.symbols.OP_REGEX_SEARCH,
                self.symbols.OP_STRICT_CONTAINS,
                self.symbols.OP_CONTAINS,
                self.symbols.OP_X_EVAL,
                self.symbols.OP_COL_EVAL,
                self.symbols.OP_IS_STR,
                self.symbols.OP_IS_INT,
                self.symbols.OP_IS_FLOAT,
                self.symbols.OP_IS_NUM,
                self.symbols.OP_IS_BOOL,
                self.symbols.OP_IS_DATE,
                self.symbols.OP_IS_DATETIME,
                self.symbols.OP_IS_ANY,
                self.symbols.OP_IS_NA,
                self.symbols.OP_IS_NK,
                self.symbols.OP_IS_YN,
                self.symbols.OP_IS_YES,
                self.symbols.OP_IS_NO,
                ],
            )

        value = text.strip()

        if operator.unary and len(value)>0:
            log(f'warning: unary operator "{operator}" cannot use a value. value "{value}" will be ignored',
                '_parse_expression', self.verbosity)
            value = ''
    
        self.expression.connector = connector
        self.expression.negate = negate
        self.expression.operator = operator
        self.expression.value = value
        self.expression.frepr = f"""
            <br>type: {self.expression.type}
            <br>text: {self.expression.text}
            <br>function: {self.expression.function}
            <br>connector: {self.expression.connector.symbol}
            <br>negate: {self.expression.negate.symbol}
            <br>operator: {self.expression.operator.symbol}
            <br>value: {self.expression.value}
            """

        log(f'debug: parsed "{self.expression.text}" as expression: {self.expression.frepr}',
            'df.q.select_cols()', self.verbosity)


        #select cols using parsed expression
    
        cols_filtered_new = self._filter_series(self.df.columns.to_series())
        self.cols_filtered = self._update_index(self.cols_filtered, cols_filtered_new, connector)

        if cols_filtered_new.any() == False:
            log(f'warning: no columns fulfill the condition in "{self.expression.text}"',
                'df.q.select_cols()', self.verbosity)


    def select_rows(self):

        #parse the expression

        connector, text = self.match_symbol(
            self.expression.text[2:],
            default=self.symbols.CONNECTOR_RESET,
            symbols=[
                self.symbols.CONNECTOR_AND,
                self.symbols.CONNECTOR_OR
                ]
            )
        
        select_which, text = self.match_symbol(
            text,
            default=self.symbols.COLS_ANY,
            symbols=[
                self.symbols.COLS_ANY,
                self.symbols.COLS_ALL,
                self.symbols.INDEX
                ]
            )

     
        negate, text = self.match_symbol(
            text,
            default=self.symbols.NEGATE_FALSE,
            symbols=[self.symbols.NEGATE_TRUE]
            )
        
        operator, text = self.match_symbol(
            text,
            default=self.symbols.OP_EQUAL,
            symbols=[
                self.symbols.OP_BIGGER_EQUAL,
                self.symbols.OP_SMALLER_EQUAL,
                self.symbols.OP_BIGGER,
                self.symbols.OP_SMALLER,
                self.symbols.OP_STRICT_EQUAL,
                self.symbols.OP_EQUAL,
                self.symbols.OP_REGEX_MATCH,
                self.symbols.OP_REGEX_SEARCH,
                self.symbols.OP_STRICT_CONTAINS,
                self.symbols.OP_CONTAINS,
                self.symbols.OP_LOAD,
                self.symbols.OP_X_EVAL,
                self.symbols.OP_COL_EVAL,
                self.symbols.OP_IS_STR,
                self.symbols.OP_IS_INT,
                self.symbols.OP_IS_FLOAT,
                self.symbols.OP_IS_NUM,
                self.symbols.OP_IS_BOOL,
                self.symbols.OP_IS_DATE,
                self.symbols.OP_IS_DATETIME,
                self.symbols.OP_IS_ANY,
                self.symbols.OP_IS_NA,
                self.symbols.OP_IS_NK,
                self.symbols.OP_IS_YN,
                self.symbols.OP_IS_YES,
                self.symbols.OP_IS_NO,
                self.symbols.OP_IS_UNIQUE,
                ],
            )

        value = text.strip()

        if operator.unary and len(value)>0:
            log(f'warning: unary operator "{operator}" cannot use a value. value "{value}" will be ignored',
                '_parse_expression', self.verbosity)
            value = ''
    
        self.expression.connector = connector
        self.expression.select_which = select_which
        self.expression.negate = negate
        self.expression.operator = operator
        self.expression.value = value
        self.expression.frepr = f"""
            <br>type: {self.expression.type}
            <br>text: {self.expression.text}
            <br>function: {self.expression.function}
            <br>connector: {self.expression.connector.symbol}
            <br>negate: {self.expression.negate.symbol}
            <br>operator: {self.expression.operator.symbol}
            <br>value: {self.expression.value}
            """

        log(f'debug: parsed "{self.expression.text}" as expression: {self.expression.frepr}',
            'df.q.select_cols()', self.verbosity)


        #select rows using parsed expression

        if self.cols_filtered.any() == False:
            log(f'warning: row filter cannot be applied when no columns where selected', 'df.q', self.verbosity)
            return
                
        if select_which == self.symbols.INDEX:
            rows_filtered_new = self._filter_series(self.df.index.to_series())
            self.rows_filtered = self._update_index(self.rows_filtered, rows_filtered_new, connector)

        else:
            rows_filtered_temp = None
            for col in self.df.columns[self.cols_filtered]:
                rows_filtered_new = self._filter_series(self.df[col])
                rows_filtered_temp = self._update_index(rows_filtered_temp, rows_filtered_new, select_which)
            self.rows_filtered = self._update_index(self.rows_filtered, rows_filtered_temp, connector)

            if rows_filtered_temp.any() == False:
                log(f'warning: no rows fulfill the condition in "{self.expression.text}"', 'df.q', self.verbosity)
            

    def _filter_series(self, series):
        operator = self.expression.operator
        value = self.expression.value
        ops = self.symbols


        if operator == ops.OP_BIGGER_EQUAL:
            filtered = pd.to_numeric(series, errors='coerce') >= pd.to_numeric(value)
        elif operator == ops.OP_SMALLER_EQUAL:
            filtered = pd.to_numeric(series, errors='coerce') <= pd.to_numeric(value)
        elif operator == ops.OP_BIGGER:
            filtered = pd.to_numeric(series, errors='coerce') > pd.to_numeric(value)
        elif operator == ops.OP_SMALLER:
            filtered = pd.to_numeric(series, errors='coerce') < pd.to_numeric(value)
            
            
        #regex comparison
        elif operator == ops.OP_REGEX_MATCH:
            filtered = series.astype(str).str.fullmatch(value) 
        elif operator == ops.OP_REGEX_SEARCH:
            filtered = series.astype(str).str.contains(value)


        #string equality comparison
        elif operator == ops.OP_STRICT_EQUAL:
            filtered = series.astype(str) == value
        elif operator == ops.OP_EQUAL:
            value_lenient = [value]
            try:
                value_lenient.append(str(float(value)))
                value_lenient.append(str(int(float(value))))
            except:
                value_lenient.append(value.lower())
            filtered = series.astype(str).str.lower().isin(value_lenient)
            
        #substring comparison
        elif operator == ops.OP_STRICT_CONTAINS:
            filtered = series.astype(str).str.contains(value, case=True, regex=False)
        elif operator == ops.OP_CONTAINS:
            filtered = series.astype(str).str.contains(value, case=False, regex=False)


        #lambda function
        elif operator == ops.OP_X_EVAL:
            filtered = series.apply(lambda x: eval(value, {'x': x, 'col': series, 'df': self.df, 'pd': pd, 'np': np, 'qp': qp}))
        elif operator == ops.OP_COL_EVAL:
            filtered = eval(value, {'col': series, 'df': self.df, 'pd': pd, 'np': np, 'qp': qp})

        #load saved selection
        elif operator == ops.OP_LOAD:
            if value in self.df.columns:
                filtered = self.df[value]
            else:
                log(f'error: column "{value}" does not exist in dataframe. cannot load selection',
                    '_filter()', self.verbosity)


        #type checks
        elif operator == ops.OP_IS_STR:
            filtered = series.apply(lambda x: isinstance(x, str))
        elif operator == ops.OP_IS_INT:
            filtered = series.apply(lambda x: isinstance(x, int))
        elif operator == ops.OP_IS_FLOAT:
            filtered = series.apply(lambda x: isinstance(x, float))
        elif operator == ops.OP_IS_NUM:
            filtered = series.apply(lambda x: qp_num(x, errors='ERROR')) != 'ERROR'
        elif operator == ops.OP_IS_BOOL:
            filtered = series.apply(lambda x: isinstance(x, bool))

        elif operator == ops.OP_IS_DATETIME:
            filtered = series.apply(lambda x: qp_datetime(x, errors='ERROR')) != 'ERROR'
        elif operator == ops.OP_IS_DATE:
            filtered = series.apply(lambda x: qp_date(x, errors='ERROR')) != 'ERROR'

        elif operator == ops.OP_IS_ANY:
            filtered = series.apply(lambda x: True)
        elif operator == ops.OP_IS_NA:
            filtered = series.apply(lambda x: qp_na(x, errors='ERROR')) != 'ERROR'
        elif operator == ops.OP_IS_NK:
            filtered = series.apply(lambda x: qp_nk(x, errors='ERROR')) != 'ERROR'
        elif operator == ops.OP_IS_YN:
            filtered = series.apply(lambda x: qp_yn(x, errors='ERROR')) != 'ERROR'
        elif operator == ops.OP_IS_YES:
            filtered = series.apply(lambda x: qp_yn(x, errors='ERROR', yes=1)) == 1
        elif operator == ops.OP_IS_NO:
            filtered = series.apply(lambda x: qp_yn(x, errors='ERROR', no=0)) == 0
        elif operator == ops.OP_IS_UNIQUE:
            filtered = series.duplicated(keep='first') == False

        else:
            log(f'error: operator "{operator}" is not implemented', '_filter()', self.verbosity)
            filtered = None


        if self.expression.negate == ops.NEGATE_TRUE:
            filtered = ~filtered

        return filtered

    def _update_index(self, values, values_new, connector):
        if values is None:
            values = values_new
        elif connector == self.symbols.CONNECTOR_RESET:
            values = values_new
        elif connector in [self.symbols.CONNECTOR_AND, self.symbols.COLS_ALL]:
            values &= values_new
        elif connector in [self.symbols.CONNECTOR_OR, self.symbols.COLS_ANY]:
            values |= values_new
        return values


    def modify_vals(self):

        #parse the expression

        connector, text = self.match_symbol(
            self.expression.text[2:],
            default=self.symbols.CONNECTOR_RESET,
            symbols=[
                self.symbols.CONNECTOR_AND,
                self.symbols.CONNECTOR_OR
            ]
            )

        operator, text = self.match_symbol(
            text,
            default=self.symbols.OP_SET_VAL,
            symbols=[
                self.symbols.OP_SET_VAL,
                self.symbols.OP_ADD_VAL,
                self.symbols.OP_SET_X_EVAL,
                self.symbols.OP_SET_COL_EVAL,
                self.symbols.OP_SET_HEADER_EVAL,
                self.symbols.OP_TO_STR,
                self.symbols.OP_TO_INT,
                self.symbols.OP_TO_FLOAT,
                self.symbols.OP_TO_NUM,
                self.symbols.OP_TO_BOOL,
                self.symbols.OP_TO_DATE,
                self.symbols.OP_TO_DATETIME,
                self.symbols.OP_TO_NA,
                self.symbols.OP_TO_NK,
                self.symbols.OP_TO_YN,
                ],
            )

        value = text.strip()

        if operator.unary and len(value)>0:
            log(f'warning: unary operator "{operator}" cannot use a value. value "{value}" will be ignored',
                'df.q.modify_vals()', self.verbosity)
            value = ''
    
        self.expression.connector = connector
        self.expression.operator = operator
        self.expression.value = value
        self.expression.frepr = f"""
            <br>type: {self.expression.type}
            <br>text: {self.expression.text}
            <br>function: {self.expression.function}
            <br>connector: {self.expression.connector.symbol}
            <br>operator: {self.expression.operator.symbol}
            <br>value: {self.expression.value}
            """

        log(f'debug: parsed "{self.expression.text}" as expression: {self.expression.frepr}',
            'df.q.modify_vals()', self.verbosity)


        #modify values using parsed expression
        ops = self.symbols
        rows = self.rows_filtered
        cols = self.cols_filtered
   
        if pd.__version__ >= '2.1.0':
            #data modification
            if operator == ops.OP_SET_VAL:
                self.df.loc[rows, cols] = value
            elif operator == ops.OP_ADD_VAL:
                self.df.loc[rows, cols] = self.df.loc[rows, cols].astype(str) + value
            
            elif operator == ops.OP_SET_X_EVAL:
                self.df.loc[rows, cols] = self.df.loc[rows, cols].map(lambda x: eval(value, {'x': x, 'df': self.df, 'pd': pd, 'np': np, 'qp': qp}))
            elif operator == ops.OP_SET_COL_EVAL:
                self.df.loc[:, cols] = self.df.loc[:, cols].apply(lambda x: eval(value, {'col': x, 'df': self.df, 'pd': pd, 'np': np, 'qp': qp}), axis=0)
            elif operator == ops.OP_SET_HEADER_EVAL:
                self.df.columns = self.df.columns.map(lambda x: eval(value, {'header': x, 'df': self.df, 'pd': pd, 'np': np, 'qp': qp}))


            #type conversion
            elif operator == ops.OP_TO_STR:
                self.df.loc[rows, cols] = self.df.loc[rows, cols].map(str)
            elif operator == ops.OP_TO_INT:
                self.df.loc[rows, cols] = self.df.loc[rows, cols].map(qp_int)
            elif operator == ops.OP_TO_FLOAT:
                self.df.loc[rows, cols] = self.df.loc[rows, cols].map(qp_float)
            elif operator == ops.OP_TO_NUM:
                self.df.loc[rows, cols] = self.df.loc[rows, cols].map(qp_num)
            elif operator == ops.OP_TO_BOOL:
                self.df.loc[rows, cols] = self.df.loc[rows, cols].map(qp_bool)
            
            elif operator == ops.OP_TO_DATETIME:
                self.df.loc[rows, cols] = self.df.loc[rows, cols].map(qp_datetime)
            elif operator == ops.OP_TO_DATE:
                self.df.loc[rows, cols] = self.df.loc[rows, cols].map(qp_date)

            elif operator == ops.OP_TO_NA:
                self.df.loc[rows, cols] = self.df.loc[rows, cols].map(qp_na)
            elif operator == ops.OP_TO_NK:
                self.df.loc[rows, cols] = self.df.loc[rows, cols].map(qp_nk)
            elif operator == ops.OP_TO_YN:
                self.df.loc[rows, cols] = self.df.loc[rows, cols].map(qp_yn)

        else:
            #data modification
            if operator == ops.OP_SET_VAL:
                self.df.loc[rows, cols] = value
            elif operator == ops.OP_ADD_VAL:
                self.df.loc[rows, cols] = self.df.loc[rows, cols].astype(str) + value
            
            elif operator == ops.OP_SET_X_EVAL:
                self.df.loc[rows, cols] = self.df.loc[rows, cols].applymap(lambda x: eval(value, {'x': x, 'df': self.df, 'pd': pd, 'np': np, 'qp': qp}))
            elif operator == ops.OP_SET_COL_EVAL:
                self.df.loc[:, cols] = self.df.loc[:, cols].apply(lambda x: eval(value, {'col': x, 'df': self.df, 'pd': pd, 'np': np, 'qp': qp}), axis=0)
            elif operator == ops.OP_SET_HEADER_EVAL:
                self.df.columns = self.df.columns.applymap(lambda x: eval(value, {'header': x, 'df': self.df, 'pd': pd, 'np': np, 'qp': qp}))


            #type conversion
            elif operator == ops.OP_TO_STR:
                self.df.loc[rows, cols] = self.df.loc[rows, cols].applymap(str)
            elif operator == ops.OP_TO_INT:
                self.df.loc[rows, cols] = self.df.loc[rows, cols].applymap(qp_int)
            elif operator == ops.OP_TO_FLOAT:
                self.df.loc[rows, cols] = self.df.loc[rows, cols].applymap(qp_float)
            elif operator == ops.OP_TO_NUM:
                self.df.loc[rows, cols] = self.df.loc[rows, cols].applymap(qp_num)
            elif operator == ops.OP_TO_BOOL:
                self.df.loc[rows, cols] = self.df.loc[rows, cols].applymap(qp_bool)
            
            elif operator == ops.OP_TO_DATETIME:
                self.df.loc[rows, cols] = self.df.loc[rows, cols].applymap(qp_datetime)
            elif operator == ops.OP_TO_DATE:
                self.df.loc[rows, cols] = self.df.loc[rows, cols].applymap(qp_date)

            elif operator == ops.OP_TO_NA:
                self.df.loc[rows, cols] = self.df.loc[rows, cols].applymap(qp_na)
            elif operator == ops.OP_TO_NK:
                self.df.loc[rows, cols] = self.df.loc[rows, cols].applymap(qp_nk)
            elif operator == ops.OP_TO_YN:
                self.df.loc[rows, cols] = self.df.loc[rows, cols].applymap(qp_yn)
      

    def new_col(self):

        #parse the expression

        connector, text = self.match_symbol(
            self.expression.text[2:],
            default=self.symbols.CONNECTOR_RESET,
            symbols=[]
            )

        operator, text = self.match_symbol(
            text,
            default=self.symbols.OP_NEW_COL_STR,
            symbols=[
                self.symbols.OP_NEW_COL_STR,
                self.symbols.OP_NEW_COL_TRUE,
                ]
            )

        value = text.strip()

        if operator.unary and len(self.value)>0:
            log(f'warning: unary operator "{operator}" cannot use a value. value "{value}" will be ignored',
                'df.q.new_col()', self.verbosity)
            value = ''
    
        self.expression.connector = connector
        self.expression.operator = operator
        self.expression.value = value
        self.expression.frepr = f"""
            <br>type: {self.expression.type}
            <br>text: {self.expression.text}
            <br>function: {self.expression.function}
            <br>connector: {self.expression.connector.symbol}
            <br>operator: {self.expression.operator.symbol}
            <br>value: {self.expression.value}
            """

        log(f'debug: parsed "{self.expression.text}" as expression: {self.expression.frepr}',
            'df.q.new_col()', self.verbosity)


        #add new column using parsed expression
        ops = self.symbols
        rows = self.rows_filtered
        cols = self.cols_filtered
        
        if operator == ops.OP_NEW_COL_STR:
            if value in self.df.columns:
                log(f'warning: column "{value}" already exists in dataframe. selecting existing col',
                    'df.q.new_col', self.verbosity)
                self.cols_filtered = pd.Index([True if col == value else False for col in self.df.columns])
            else:
                self.df[value] = ''
                self.cols_filtered = pd.Index([True if col == value else False for col in self.df.columns])
        if operator == ops.OP_NEW_COL_TRUE:
            if value in self.df.columns:
                log(f'warning: column "{value}" already exists in dataframe. selecting existing col and resetting values',
                    'df.q.new_col', self.verbosity)
                self.df[value] = self.rows_filtered
                self.cols_filtered = pd.Index([True if col == value else False for col in self.df.columns])
            else:
                self.df[value] = self.rows_filtered
                self.cols_filtered = pd.Index([True if col == value else False for col in self.df.columns])
                       



@pd.api.extensions.register_dataframe_accessor('qi')
class DataFrameQueryInteractiveMode:
    """
    Wrapper for df.q() for interactive use in Jupyter notebooks.
    """
    def __init__(self, df: pd.DataFrame):
        self.df = df

    def __call__(self, num_filters=5):
        kwargs = {'df': fixed(self.df)}

        #code input
        ui_code = widgets.Textarea(
            value='',
            placeholder='Enter query code here',
            layout=Layout(width='95%', height='100%')
        )
        kwargs['code'] = ui_code



        ui_diff = widgets.ToggleButtons(
            options=[None, 'mix', 'old', 'new', 'new+'],
            description='show differences mode:',
            tooltips=[
                'dont show differences, just show the new (filtered) dataframe.',
                'show new (filtered) dataframe plus all the removed (filtered) values from the old dataframe. values affected by the filters are marked green (newly added), yellow (modified), red (deleted)',
                'show old (unfiltered) dataframe. values affected by the filters are marked green (newly added), yellow (modified), red (deleted)',
                'show new (filtered) dataframe. values affected by the filters are marked green (newly added), yellow (modified), red (deleted)',
                'show new (filtered) dataframe but also adds metadata columns with the prefix "#". If a value changed, the metadata column contains the old value. values affected by the filters are marked green (newly added), yellow (modified), red (deleted)',
                ],
            )
        kwargs['diff'] = ui_diff

        ui_verbosity = widgets.ToggleButtons(
            options=[0, 1, 2, 3, 4, 5],
            value=3,
            description='verbosity level:',
            tooltips=[
                'no logging',
                'only errors',
                'errors and warnings',
                'errors, warnings and info',
                'errors, warnings, info and debug',
                'errors, warnings, info, debug and trace',
                ],
            )
        kwargs['verbosity'] = ui_verbosity

        ui_inplace = widgets.ToggleButtons(
            options=[True, False],
            value=False,
            description='make modifications inplace:',
            tooltips=[
                'make modifications inplace, e.g. change the original dataframe.',
                'return a new dataframe with the modifications. lower performance.',
                ],
            )
        kwargs['inplace'] = ui_inplace


        ui_settings = VBox([
            ui_diff,
            ui_verbosity,
            ui_inplace,
            ])
        

        ui_help = widgets.Tab(
            children=[
                ui_settings,
                widgets.HTML(value="syntax"),
                widgets.HTML(value="operators"),
                widgets.HTML(value="modifiers"),
                ],
            titles=['settings', 'syntax', 'operators', 'modifiers'],
            layout=Layout(width='50%', height='95%')
            )
        

        ui_input = VBox([ui_code], layout=Layout(width='50%', height='100%'))
        ui = HBox([ui_input, ui_help], layout=Layout(width='100%', height='300px'))

        display(ui)
        out = HBox([interactive_output(_interactive_mode, kwargs)], layout=Layout(overflow_y='auto'))
        display(out)


def _interactive_mode(**kwargs):

    df = kwargs.pop('df')

    result = df.q(
        code=kwargs['code'],
        inplace=kwargs['inplace'],
        diff=kwargs['diff'],
        verbosity=kwargs['verbosity'],
        # max_cols=kwargs['max_cols'],
        # max_rows=kwargs['max_rows'],
        )


    
    display(result)
    print('input code: ', df.qp._input)
    return result 








# if 'cards' not in globals():
#     cards = pd.read_csv('data/cards.csv')

df = qp.get_df()
# df['bad /name´s'] = None


# cards.q('toughness ´r >2 & <5')


df.q(
    r"""

    gender ´m to str & x= x.lower()
    gender ´r is unique

    is any


    """,
    diff='mix',
    inplace=False,
    verbosity=5,
    )


C:\Users\MartinVölkl-GouyaIns\AppData\Local\Temp\ipykernel_37304\1624087641.py:143: UserWarning: registration of accessor <class '__main__.DataFrameQuery'> under name 'q' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  @pd.api.extensions.register_dataframe_accessor('q')
C:\Users\MartinVölkl-GouyaIns\AppData\Local\Temp\ipykernel_37304\1624087641.py:874: UserWarning: registration of accessor <class '__main__.DataFrameQueryInteractiveMode'> under name 'qi' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  @pd.api.extensions.register_dataframe_accessor('qi')


183,df was checked. no problems found,qp.pd_util._check_df,info,2024-07-25 11:08:44.468516


184,"found symbol ""Symbol('´c': 'default. select columns by a condition)'"" in string ""´c gender""",_read_symbol,trace,2024-07-25 11:08:44.474517


185,"no symbol found in string ""gender"". using default ""Symbol('': 'default. only the current condition must be fulfilled)'""",_read_symbol,trace,2024-07-25 11:08:44.480794


186,"no symbol found in string ""gender"". using default ""Symbol('': 'default. dont negate the condition)'""",_read_symbol,trace,2024-07-25 11:08:44.487604


...

212,"parsed ""´c is any"" as expression: type: Symbol('´c': 'default. select columns by a condition)' text: ´c is any function: select_cols connector: negate: operator: is any value:",df.q.select_cols(),debug,2024-07-25 11:08:44.777943


213,"adding column ""meta"" at position 0",df.format(),info,2024-07-25 11:08:44.789104


214,"adding column ""meta"" at position 0",df.format(),info,2024-07-25 11:08:44.843831


{'rows removed': 2}

,meta,ID,name,date of birth,age,gender,height,weight,bp systole,bp diastole,cholesterol,diabetes,dose
0,,10001.000000,John Doe,1995-01-02,-25,m,170,70.200000,20,80,Normal,No,10kg
1,,10002.000000,Jane Smith,1990/09/14,30,f,175.5cm,68,130,85,Highe,yes,NaN
2,,10003.000000,Alice Johnson,1985.08.23,nan,female,None,72.5lb,NaN,nan,NaN,N/A,15 mg once a day
3,,20001.000000,Bob Brown,19800406,None,male,280,na,140,90mmHg,GOOD,No,20mg
4,,20002.000000,eva white,05-11-2007,40.0,other,NaN,,135mmhg,nan,n.a.,Y,20 Mg
6,,30001.000000,Grace TAYLOR,28-05-1975,nan,ff,1,None,NAN,NaN,Normal,NO,NaN
7,,30002.000000,Harry Clark,1960Mar08,unk,nan,6ft 1in,80.3,122,None,n/a,None,None
8,,30003.000000,IVY GREEN,1955-Jan-09,,none,-10,130lbs,,95,high,NaN,30 MG
9,,30004.000000,JAck Williams,1950 Sep 10,unknown,mal,,82,130,0,,n,35
5,removed row,20003.000000,Frank miller,06-30-1983,forty-five,m,185,75kg,125,75,High,Yes,25g


In [28]:
#sort df by name
df = qp.get_df()
# df = df.sort_values('name')
df

# df = df.sort_values('diabetes')

df.loc[5:, :].sort_values('diabetes', inplace=True)

df

C:\Users\MartinVölkl-GouyaIns\AppData\Local\Temp\ipykernel_37304\1001983613.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[5:, :].sort_values('diabetes', inplace=True)


,ID,name,date of birth,age,gender,height,weight,bp systole,bp diastole,cholesterol,diabetes,dose
0,10001,John Doe,1995-01-02,-25,M,170,70.2,20,80,Normal,No,10kg
1,10002,Jane Smith,1990/09/14,30,F,175.5cm,68,130,85,Highe,yes,NaN
2,10003,Alice Johnson,1985.08.23,NaN,Female,None,72.5lb,NaN,nan,NaN,N/A,15 mg once a day
3,20001,Bob Brown,19800406,None,Male,280,na,140,90mmHg,GOOD,No,20mg
4,20002,eva white,05-11-2007,40.0,Other,NaN,,135mmhg,NaN,n.a.,Y,20 Mg
5,20003,Frank miller,06-30-1983,forty-five,m,185,75kg,125,75,High,Yes,25g
6,30001,Grace TAYLOR,28-05-1975,nan,ff,1,None,NAN,NaN,Normal,NO,NaN
7,30002,Harry Clark,1960Mar08,unk,NaN,6ft 1in,80.3,122,None,n/a,None,None
8,30003,IVY GREEN,1955-Jan-09,,None,-10,130lbs,,95,high,NaN,30 MG
9,30004,JAck Williams,1950 Sep 10,unknown,Mal,,82,130,0,,n,35


In [13]:
cards = pd.read_csv('data/cards.csv')



C:\Users\MartinVölkl-GouyaIns\AppData\Local\Temp\ipykernel_22748\1002163129.py:1: DtypeWarning: Columns (2,3,7,12,16,20,23,47,52,53,61,62,66,67) have mixed types. Specify dtype option on import or set low_memory=False.
  cards = pd.read_csv('data/cards.csv')


In [19]:
cards.q(
    r"""
    
    # faceConvertedManaCost  ´r 2
    # power ´r >3

    name / text / type / power / toughness / manaCost / subtypes
    
    """,
    diff=None,
    inplace=False,
    verbosity=3,
    )

,manaCost,name,power,subtypes,text,toughness,type
0,{5}{W}{W},Ancestor's Chosen,4,"Human, Cleric",First strike (This creature deals combat damag...,4,Creature — Human Cleric
1,{5}{W}{W},Ancestor's Chosen,4,"Human, Cleric",First strike (This creature deals combat damag...,4,Creature — Human Cleric
2,{4}{W},Angel of Mercy,3,Angel,Flying\nWhen Angel of Mercy enters the battlef...,3,Creature — Angel
3,{4}{W},Angel of Mercy,3,Angel,Flying\nWhen Angel of Mercy enters the battlef...,3,Creature — Angel
4,{2}{W},Angelic Blessing,NaN,NaN,Target creature gets +3/+3 and gains flying un...,NaN,Sorcery
...,...,...,...,...,...,...,...
83227,{1}{U},Into the Roil,NaN,NaN,Kicker {1}{U} (You may pay an additional {1}{U...,NaN,Instant
83228,{B},Bloodchief's Thirst,NaN,NaN,Kicker {2}{B} (You may pay an additional {2}{B...,NaN,Sorcery
83229,{1}{R},Roil Eruption,NaN,NaN,Kicker {5} (You may pay an additional {5} as y...,NaN,Sorcery
83230,{2}{G},Roiling Regrowth,NaN,NaN,Sacrifice a land. Search your library for up t...,NaN,Instant


# qp.diff()

In [20]:

import pandas as pd
import numpy as np
import copy
import os
import datetime

from IPython.display import display
from ipywidgets import interact, widgets
from pandas.api.extensions import register_dataframe_accessor

from qplib.pd_util import _format_df, get_dfs
from qplib.util import log, GREEN, RED, ORANGE, GREEN_LIGHT, RED_LIGHT, ORANGE_LIGHT
from qplib.types import qp_date, qp_na, qpDict




df_new, df_old = get_dfs()

_show_differences(df_new, df_old, show='new+', max_cols=200, max_rows=20, verbosity=3)




34,"adding column ""meta"" at position 0",df.format(),info,2024-07-17 12:26:24.573959


35,"adding column ""meta"" at position 0",df.format(),info,2024-07-17 12:26:24.587968


{'cols added': 1,
 'cols removed': 1,
 'rows added': 1,
 'rows removed': 1,
 'vals added': 1,
 'vals removed': 1,
 'vals changed': 1}

,meta,d,old: d,b,old: b,a,old: a
y,vals changed: 1,2.000000,,2.000000,,0.000000,2.000000
x2,added row,1.000000,,1.000000,,1.000000,
z,vals added: 1vals removed: 1,3.000000,,3.000000,None,nan,3.000000


# excel_diff()

In [94]:

import pandas as pd
import numpy as np
import copy
import os
import datetime

from IPython.display import display
from ipywidgets import interact, widgets
from pandas.api.extensions import register_dataframe_accessor

from qplib.pd_util import _format_df, get_dfs
from qplib.util import log, GREEN, RED, ORANGE, GREEN_LIGHT, RED_LIGHT, ORANGE_LIGHT
from qplib.types import qp_date, qp_na, qpDict





# file_new = 'archive/stats_new.xlsx'
# file_old = 'archive/stats_old.xlsx'

# summary, results = excel_diff('archive/stats_new.xlsx', 'archive/stats_old.xlsx', to_excel=True)

new, old = qp.get_dfs()
display(new, old)
_show_differences(new, old, show='new+')

,d,b,a
y,2.0,2.0,0.0
x2,1.0,1.0,1.0
z,3.0,3.0,NaN


,a,b,c
x,1.0,1.0,1.0
y,2.0,2.0,2.0
z,3.0,None,3.0


343,"adding column ""meta"" at position 0",df.format(),info,2024-07-17 15:10:13.172466


344,"adding column ""meta"" at position 0",df.format(),info,2024-07-17 15:10:13.180861


{'cols added': 1,
 'cols removed': 1,
 'rows added': 1,
 'rows removed': 1,
 'vals added': 1,
 'vals removed': 1,
 'vals changed': 1}

,meta,d,old: d,b,old: b,a,old: a
y,vals changed: 1,2.000000,,2.000000,,0.000000,2.000000
x2,added row,1.000000,,1.000000,,1.000000,
z,vals added: 1vals removed: 1,3.000000,,3.000000,None,nan,3.000000


In [90]:
new, old = qp.get_dfs()
display(new, old)
_show_differences(new, old, show='new+')

,d,b,a
y,2.0,2.0,0.0
x2,1.0,1.0,1.0
z,3.0,3.0,NaN


,a,b,c
x,1.0,1.0,1.0
y,2.0,2.0,2.0
z,3.0,None,3.0


335,"adding column ""meta"" at position 0",df.format(),info,2024-07-17 14:50:32.925219


336,"adding column ""meta"" at position 0",df.format(),info,2024-07-17 14:50:32.931791


{'cols added': 1,
 'cols removed': 1,
 'rows added': 1,
 'rows removed': 1,
 'vals added': 1,
 'vals removed': 1,
 'vals changed': 1}

,meta,d,old: d,b,old: b,a,old: a
y,vals changed: 1,2.000000,,2.000000,,0.000000,2.000000
x2,added row,1.000000,,1.000000,,1.000000,
z,vals added: 1vals removed: 1,3.000000,,3.000000,None,nan,3.000000


# pd_util

In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import datetime
import qplib as qp

from IPython.display import display
from ipywidgets import interact, widgets
from pandas.api.extensions import register_dataframe_accessor

from qplib.util import log, qpDict
from qplib.types import qp_date, qp_na






df = pd.DataFrame({
    'ID': [10001, 10002, 10003, 20001, 20002, 20003, 30001, 30002, 30003, 30004, 30005],
    'name': ['John Doe', 'Jane Smith', 'Alice Johnson', 'Bob Brown', 'eva white', 'Frank miller', 'Grace TAYLOR', 'Harry Clark', 'IVY GREEN', 'JAck Williams', 'john Doe'],
    'date of birth': ['1995-01-02', '1990/09/14', '1985.08.23', '19800406', '05-11-2007', '06-30-1983', '28-05-1975', '1960Mar08', '1955-Jan-09', '1950 Sep 10', '1945 October 11'],
    'age': [-25, '30', np.nan, None, '40.0', 'forty-five', 'nan', 'unk', '', 'unknown', 35],
    'gender': ['M', 'F', 'Female', 'Male', 'Other', 'm', 'ff', 'NaN', None, 'Mal', 'female'],
    'height': [170, '175.5cm', None, '280', 'NaN', '185', '1', '6ft 1in', -10, '', 200],
    'weight': [70.2, '68', '72.5lb', 'na', '', '75kg', None, '80.3', '130lbs', '82', -65],
    'bp systole': ['20', 130, 'NaN', '140', '135mmhg', '125', 'NAN', '122', '', 130, '45'],
    'bp diastole': [80, '85', 'nan', '90mmHg', np.nan, '75', 'NaN', None, '95', '0', 'NaN'],
    'cholesterol': ['Normal', 'Highe', 'NaN', 'GOOD', 'n.a.', 'High', 'Normal', 'n/a', 'high', '', 'Normal'],
    'diabetes': ['No', 'yes', 'N/A', 'No', 'Y', 'Yes', 'NO', None, 'NaN', 'n', 'Yes'],
    'dose': ['10kg', 'NaN', '15 mg once a day', '20mg', '20 Mg', '25g', 'NaN', None, '30 MG', '35', '40ml']
    })

df_new, df_old = qp.get_dfs()






# types

In [ ]:
import pandas as pd
import numpy as np
import re


def qp_int(x, errors='coerce', na=np.nan):
    try:
        return int(float(x))  #float first to handle strings like '1.0'
    except:
        match errors:
            case 'raise':
                raise ValueError(f"""could not convert "{x}" to integer.
                    Error handling:
                    errors='ignore': returns the original value
                    errors='raise': raises a ValueError
                    errors='coerce': returns np.nan
                    errors=<any other value>: returns <any other value>
                    """)
            case 'ignore':
                return x
            case 'coerce':
                return na
            case _:
                return errors

def qp_float(x, errors='coerce', na=np.nan):
    try:
        return float(x)
    except:
        match errors:
            case 'raise':
                raise ValueError(f"""could not convert "{x}" to float.
                    Error handling:
                    errors='ignore': returns the original value
                    errors='raise': raises a ValueError
                    errors='coerce': returns np.nan
                    errors=<any other value>: returns <any other value>
                    """)
            case 'ignore':
                return x
            case 'coerce':
                return na
            case _:
                return errors
            
def qp_num(x, errors='coerce', na=np.nan):
    try:
        return pd.to_numeric(x)
    except:
        match errors:
            case 'raise':
                raise ValueError(f"""could not convert "{x}" to numeric.
                    Error handling:
                    errors='ignore': returns the original value
                    errors='raise': raises a ValueError
                    errors='coerce': returns np.nan
                    errors=<any other value>: returns <any other value>
                    """)
            case 'ignore':
                return x
            case 'coerce':
                return na
            case _:
                return errors
            
def qp_bool(x, errors='coerce', na=None):
    if str(x).lower() in ['y', 'yes', 'true', '1', '1.0', 'positive', 'pos']:
        return True
    elif str(x).lower() in ['n', 'no', 'false', '0', '0.0', 'negative', 'neg']:
        return False
    else:
        match errors:
            case 'raise':
                raise ValueError(f"""could not convert "{x}" to boolean.
                    Error handling:
                    errors='ignore': returns the original value
                    errors='raise': raises a ValueError
                    errors='coerce': returns NaN
                    errors=<any other value>: returns <any other value>
                    """)
            case 'ignore':
                return x
            case 'coerce':
                return na
            case _:
                return errors


def qp_date(x, errors='coerce', na=pd.NaT):
    if isinstance(x, str):
        x = x.replace('_', '-')
    try:
        if re.match(r'\D*(1|2)\d\d\d', x):
            return pd.to_datetime(x, dayfirst=False).date()
        else:
            return pd.to_datetime(x, dayfirst=True).date()
    except:
        match errors:
            case 'raise':
                raise ValueError(f"""could not convert "{x}" to date.
                    Error handling:
                    errors='ignore': returns the original value
                    errors='raise': raises a ValueError
                    errors='coerce': returns NaT
                    errors=<any other value>: returns <any other value>
                    """)
            case 'ignore':
                return x
            case 'coerce':
                return na
            case _:
                return errors
       
def qp_datetime(x, errors='coerce', na=pd.NaT):
    if isinstance(x, str):
        x = x.replace('_', '-')
    try:
        if re.match(r'\D*(1|2\d\d\d)', x):
            return pd.to_datetime(x, dayfirst=False)
        else:
            return pd.to_datetime(x, dayfirst=True)
    except:
        match errors:
            case 'raise':
                raise ValueError(f"""could not convert "{x}" to datetime.
                    Error handling:
                    errors='ignore': returns the original value
                    errors='raise': raises a ValueError
                    errors='coerce': returns NaT
                    errors=<any other value>: returns <any other value>
                    """)
            case 'ignore':
                return x
            case 'coerce':
                return na
            case _:
                return errors


def qp_na(x, errors='ignore', na=None):
    possible_nas = [
        'not available', 'na', 'n/a', 'n.a', 'n.a.', 'na.', 'n.a',
        'not a number', 'nan',
        'null', 'nil',
        'none',
        '',
        ]
    
    if pd.isna(x) or str(x).lower() in possible_nas:
        return na
    else:
        match errors:
            case 'raise':
                raise ValueError(f"""could not convert "{x}" to "{na}".
                    Error handling:
                    errors='ignore': returns the original value
                    errors='raise': raises a ValueError
                    errors='coerce': returns NaN
                    errors=<any other value>: returns <any other value>
                    """)
            case 'ignore':
                return x
            case 'coerce':
                return None
            case _:
                return errors

def qp_nk(x, errors='ignore', nk='unknown'):
    possible_nks = [
        'unk', 'unknown', 'not known', 'not known.',
        'nk', 'n.k.', 'n.k', 'n/k',
        'not specified', 'not specified.',
        ]
    
    if str(x).lower() in possible_nks:
        return nk
    else:
        match errors:
            case 'raise':
                raise ValueError(f"""could not convert "{x}" to "{nk}".
                    Error handling:
                    errors='ignore': returns the original value
                    errors='raise': raises a ValueError
                    errors='coerce': returns NaN
                    errors=<any other value>: returns <any other value>
                    """)
            case 'ignore':
                return x
            case 'coerce':
                return None
            case _:
                return errors

def qp_yn(x, errors='coerce', yes='yes', no='no', na=None):
    if str(x).lower() in ['y', 'yes', 'true', '1', '1.0', 'positive', 'pos']:
        return yes
    elif str(x).lower() in ['n', 'no', 'false', '0', '0.0', 'negative', 'neg']:
        return no
    else:
        match errors:
            case 'raise':
                raise ValueError(f"""could not convert "{x}" to "{yes}" or "{no}".
                    Error handling:
                    errors='ignore': returns the original value
                    errors='raise': raises a ValueError
                    errors='coerce': returns NaN
                    errors=<any other value>: returns <any other value>
                    """)
            case 'ignore':
                return x
            case 'coerce':
                return na
            case _:
                return errors

        




In [ ]:
import re

# Define the regex pattern to match a sequence of numbers allowing for different kinds of separators
pattern = r"\d+([,.\s_]\d+)*"

# Example string to match against
example_string = "123,456.789 0123\t456_789"

# Perform the search
matches = re.findall(pattern, example_string)

print(matches)

# "bashlike" wrappers

# temp